In [1]:
import deap
print(deap.__version__)


1.4


In [ ]:
from deap import base, creator, tools
import random

creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # maximize volume
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -1, 1)
# TODO: Yitong I don't think it's just 30 floats... it should be more!
# We have a flat 50 different landmark points... each with 3 variables - the direction, for a total of 150
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=150)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [ ]:
import subprocess

def generate_mesh(individual, output_path="cup.stl"):
    # Save points to file or pass as args
    args = [str(x) for x in individual]
    subprocess.run(["blender", "--background", "--python", "generate_cup.py", "--"] + args)


In [ ]:
import trimesh

def estimate_cuppiness(stl_path):
    mesh = trimesh.load_mesh(stl_path)
    if not mesh.is_watertight:
        return 0.0  # Leaky = not a cup

    # Axis-aligned bounding box and flood-fill assumption
    z_dir = [0, 0, 1]
    water_plane_height = mesh.bounds[1][2]  # top z
    volume = mesh.volume  # crude, but fast

    return volume  # or something fancier like inner cavity only
